# **Pràctica 2 - L'enfonsament del Titànic i les possiblitats de sobreviure**
## Inteligència Artificial - Machine Learning
### Grau en Enginyeria Informàtica - Universitat de les Illes Balears

#### Autors: Lluis Barca Pons i Victor Canelo Galera

Desde l'assignatura d'Intelgència Artificial se'ns ha demanat realitzar un anàlisis sobre el famós enfonsament del vaixell Titànic, l'any 1912. Durant aques naufragi, van morir 1502 de 2224 persones registrades, entre passatgers i tripulants.

El més interessant és que sembla ser que existia diferents probabilitats de sobreviure entre alguns grups de persones. Per tant, l'objectiu d'aquesta pràctica és realitzar un model predictiu, utilizant les dades reals recollides, que ens mostri quins tipus de persones tenien més probabiitats de viure.

Llavors, la practica es dividira en dues parts:

1. Entrenar una serie de predictors i comparar el seu rendiment.

2. Obtenir i analitzar la importancia de cada caracteristica per cada un dels models generats.

# 1.Configuració del nostre entorn

A continuació tenim els corresponents imports de les diferents llibreries de Python que utilitzarem durant tota la pràctica.

In [238]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.ensemble import RandomForestClassifier 

## 1.1. Dataset

In [239]:
df = pd.read_csv("dades.csv")
df


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


## 1.2. Neteja del Dataset

A continuació descartarem algunes variables que no ens aporten cap informació rellevant sobre la probabilitat de sobreviure. 

- PassengerId: ID del passatjer
- Name: El nom de cada tripulant
- SibSp: Germans / cònjuges pròxims a bord 
- Parch: Pares / Fills a bord
- Ticket: El nombre del ticket
- Fare: Tarifa pagada
- Embarked: Port d'embarcament

A continuació, observam les variables que contenguin valors no numèrics:

In [240]:
df.isnull().any()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked        True
dtype: bool

Ens adonam que tres de les dotze variables tenen valors no numèrics. Però, ens interessa saber quin tant per cent de les files no ens donen cap informació.

In [241]:
na_ratio = ((df.isnull().sum() / len(df))*100).sort_values(ascending = False)
print(na_ratio)

Cabin          77.104377
Age            19.865320
Embarked        0.224467
PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
dtype: float64


Llavors, la variable Cabin, tampoc serà adequada per a la realització del model; ja que el nombre de valors no numèrics és molt elevat. No obstant, la variable Age també ens mostra un tant per cent notable de nombres no numèrics. Però, aquesta variable ens dona molta informació i podem substituir les dades no numeriques per la mitjana d'edat.

In [242]:
model = df[["Survived","Pclass", "Sex", "Age"]]

# Calculam la mitjana d'edat de la gent del vaixell
edat = df[["Age"]]
edat = edat.dropna()
edat = int(edat.mean())
edat

model["Age"] = model["Age"].replace(np.nan, str(edat))
# model["Age"] = model["Age"].fillna(str(edat))

C:\Users\luisb\AppData\Local\Temp\ipykernel_9088\4171442589.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model["Age"] = model["Age"].replace(np.nan, str(edat))


A continuació, modificarem el valor del sexe per Male = 0 i Female = 1; per poder entrenar el nostre model.

In [243]:
model = model.replace(to_replace="male", value="0")
model = model.replace(to_replace="female", value="1")

Una vegada feta la neteja, ens quedaria el següent dataset:

In [244]:
model

,Survived,Pclass,Sex,Age
0,0,3,0,22.0
1,1,1,1,38.0
2,1,3,1,26.0
3,1,1,1,35.0
4,0,3,0,35.0
...,...,...,...,...
886,0,2,0,27.0
887,1,1,1,19.0
888,0,3,1,29
889,1,1,0,26.0


# 2. Entrenament

## 2.1. Regressió Logística

## 2.2. Perceptró

## 2.3. Arbres de decissió

## 2.4. Comparació dels rendiments

# 3. Anàlisis de les característiques

# 4. Conclusions